# Test the new classes for storing and querying GP samples.

## Setup

In [1]:
import os
import sys

os.chdir("..")

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src import load, munge
import src.models.gp

In [3]:
import importlib
importlib.reload(src.load)
importlib.reload(src.munge)
importlib.reload(src.models.gp)

<module 'src.models.gp' from '/Users/yl3/github/dota2/src/models/gp.py'>

## Load data.

### Load matches.

In [4]:
matches = load.all_matches_df()

In [5]:
matches_ti9 = matches.loc[matches.league_name == "The International 2019"]
matches_ti9.shape

(268, 15)

In [6]:
players_mat_ti9 = munge.make_match_players_matrix(
    matches_ti9.radiant_players, matches_ti9.dire_players)
players_mat_ti9.shape

(268, 240)

### Load players.

In [7]:
players = src.munge.match_df_to_player_df(matches_ti9)

In [8]:
players.loc[players_mat_ti9.columns[:5]]

,name,team,team_id
3916428,EGM,Bald,7237270
3940262,March,Team Jinesbrus,7225724
4281729,syndereN,Anti-MagE-,7247587
6922000,pieliedie,Forward Gaming,6214538
10366616,Sneyking,Forward Gaming,6214538


## Fit the model.

In [9]:
two_years_in_ms = 2 * 365 * 24 * 60 * 60 * 1000

In [10]:
gp = src.models.gp.SkillsGP(
    players_mat_ti9,
    matches_ti9.startTimestamp.values,
    matches_ti9.radiantVictory,
    players.loc[players_mat_ti9.columns].name,
    "exponential",
    {"scale": two_years_in_ms},
    propose_sd=0.1,
    radi_prior_sd=100,  # Non-informative prior
    logistic_scale=1,
    save_every_n_iter=10
)

### Fit numerically using Newton-CG.

In [11]:
ti9_res = gp.fit()

Starting...
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Optimization terminated successfully.
         Current function value: -7251.013163
         Iterations: 16
         Function evaluations: 18
         Gradient evaluations: 33
         Hessian evaluations: 2154


In [12]:
ti9_res

     fun: -7251.013162612677
     jac: array([-7.38146256e-06, -5.53450919e-06, -6.69449200e-06, ...,
        2.25451113e-05,  1.21444848e-06,  5.19662969e-06])
 message: 'Optimization terminated successfully.'
    nfev: 18
    nhev: 2154
     nit: 16
    njev: 33
  status: 0
 success: True
       x: array([-0.28622646, -0.28618455, -0.28613563, ..., -0.16674435,
       -0.16678095, -0.08426547])

Check radiant advantage:

In [14]:
src.models.gp.win_prob(ti9_res.x[-1], 1)

0.4789460879638722

In [15]:
skills_by_player = [ti9_res.x[start:end]
                    for start, end in gp.samples.skill_vec_idx_of_player]
skills_by_player[-1]

array([-0.16469034, -0.16480311, -0.16502255, -0.16655579, -0.16665908,
       -0.16674435, -0.16678095])

In [16]:
gp.samples.add_sample(-1, skills_by_player, ti9_res.x[-1], ti9_res.fun)

In [17]:
sample_set = gp.samples

### `GPSampleSet.player_skill_by_sample()`

In [18]:
player_skill_by_sample = sample_set.player_skill_by_sample("pieliedie", [-1])

In [19]:
player_skill_by_sample

matchId,4904311356,4904261164,4904218149,4904171017,4901998619,4901925673,4901799568,4901733286,4899990684,4899794936,4898286553,4898190830,4898118360,4898062038,4897799006
iter,,,,,,,,,,,,,,,
-1,0.596079,0.596096,0.596085,0.596053,0.597558,0.597604,0.59766,0.597679,0.597987,0.597987,0.597966,0.597947,0.597929,0.597918,0.597868


In [20]:
player_skill_by_sample.shape

(1, 15)

### Average skill of player across matches and samples.

In [21]:
skills_by_player = pd.Series(
    [np.mean(sample_set.player_skill_by_sample(player, [-1]).values)
     for player in sample_set.players.index],
    index=sample_set.players.index,
    name="skill")
print(skills_by_player.sort_values(ascending=False).to_string())

6922000      0.597361
86726887     0.597361
221666230    0.597361
108452107    0.597361
10366616     0.597361
153836240    0.403248
157989498    0.403248
119631156    0.403248
292921272    0.403248
164685175    0.403248
86725175     0.305310
84429681     0.305310
94786276     0.305310
87177591     0.305310
87196890     0.305310
72312627     0.259674
87382579     0.259674
169025618    0.259674
142139318    0.259674
98172857     0.259674
277742444    0.250589
342340720    0.250589
137129583    0.250589
163587550    0.250589
107081378    0.250589
234237646    0.248923
94004717     0.248923
87063175     0.248923
105045291    0.248923
85937380     0.248923
127565532    0.244829
480412663    0.244829
107023378    0.244829
99155835     0.244829
90324740     0.244829
86715129     0.240214
31818853     0.240214
131706718    0.240214
115141430    0.240214
349310876    0.240214
119576842    0.218784
139822354    0.218784
186627166    0.218784
89423756     0.218784
148215639    0.218784
412753955 

In [22]:
team_skills = (players.assign(skill=skills_by_player)
               .sort_values("skill")
               .groupby("team").agg({"skill": "sum"})
               .sort_values("skill", ascending=False))
team_skills

,skill
team,
Forward Gaming,2.986805
Infamous Gaming,2.016241
J.Storm,1.526552
Chaos Esports Club,1.298370
CDEC,1.252944
paiN Gaming,1.244613
Vega Squadron,1.224145
beastcoast,1.201069
Royal Never Give Up,1.093921


In [30]:
from scipy.spatial.distance import squareform, pdist

In [31]:
upper = np.triu(squareform(pdist(team_skills, metric=lambda x, y: 1 / (1 + np.exp((y - x))))))
lower = np.tril(squareform(pdist(team_skills, metric=lambda x, y: 1 / (1 + np.exp((x - y))))))

pd.DataFrame(
    upper + lower,
    index=team_skills.index,
    columns=team_skills.index).iloc[:15, :15]

team,Forward Gaming,Infamous Gaming,J.Storm,Chaos Esports Club,CDEC,paiN Gaming,Vega Squadron,beastcoast,Royal Never Give Up,Mineski,Natus Vincere,Team Jinesbrus,The Final Tribe,Hippomaniacs,Winstrike Team
team,,,,,,,,,,,,,,,
Forward Gaming,0.000000,0.725232,0.811571,0.844018,0.849906,0.850965,0.853543,0.856404,0.869084,0.871377,0.878095,0.882524,0.899034,0.915805,0.919520
Infamous Gaming,0.274768,0.000000,0.620033,0.672138,0.682069,0.683873,0.688281,0.693210,0.715515,0.719630,0.731835,0.740004,0.771353,0.804728,0.812338
J.Storm,0.188429,0.379967,0.000000,0.556799,0.567978,0.570021,0.575031,0.580660,0.606502,0.611338,0.625805,0.635596,0.673988,0.716348,0.726231
Chaos Esports Club,0.155982,0.327862,0.443201,0.000000,0.511355,0.513436,0.518548,0.524306,0.550935,0.555954,0.571037,0.581301,0.622012,0.667797,0.678612
CDEC,0.150094,0.317931,0.432022,0.488645,0.000000,0.502083,0.507199,0.512966,0.539672,0.544713,0.559875,0.570206,0.611274,0.657644,0.668626
paiN Gaming,0.149035,0.316127,0.429979,0.486564,0.497917,0.000000,0.505117,0.510884,0.537602,0.542646,0.557821,0.568163,0.609293,0.655766,0.666777
Vega Squadron,0.146457,0.311719,0.424969,0.481452,0.492801,0.494883,0.000000,0.505769,0.532510,0.537562,0.552767,0.563134,0.604410,0.651131,0.662214
beastcoast,0.143596,0.306790,0.419340,0.475694,0.487034,0.489116,0.494231,0.000000,0.526762,0.531821,0.547056,0.557449,0.598879,0.645871,0.657033
Royal Never Give Up,0.130916,0.284485,0.393498,0.449065,0.460328,0.462398,0.467490,0.473238,0.000000,0.505077,0.520397,0.530877,0.572889,0.620998,0.632499


### `GPSampleSet.team_skill_by_sample()`

In [32]:
radi_skill = sample_set.team_skill_by_sample().iloc[[-1]]
radi_skill

matchId,4904311356,4904261164,4904218149,4904171017,4904062522,4903974414,4903679968,4903566049,4903441151,4903243468,...,4889163811,4889115579,4889038294,4888974952,4888936676,4888859291,4888772755,4888732113,4888686136,4888646940
iter,,,,,,,,,,,,,,,,,,,,,
-1,1.529711,2.980478,1.530263,2.980266,1.189661,1.190033,1.329758,0.785899,1.329332,1.328472,...,1.047062,-0.182781,0.987495,-0.182042,1.046438,-0.780891,-1.399753,-0.781093,0.344864,-0.180503


In [33]:
dire_skill = sample_set.team_skill_by_sample("dire").iloc[[-1]]
dire_skill

matchId,4904311356,4904261164,4904218149,4904171017,4904062522,4903974414,4903679968,4903566049,4903441151,4903243468,...,4889163811,4889115579,4889038294,4888974952,4888936676,4888859291,4888772755,4888732113,4888686136,4888646940
iter,,,,,,,,,,,,,,,,,,,,,
-1,2.980396,1.529936,2.980427,1.530703,1.530612,1.530264,0.785651,1.329623,0.786292,0.587611,...,0.345434,0.275433,-0.292452,-1.399745,0.275238,0.345237,0.275338,-0.29183,0.987931,1.044913


In [34]:
skill_diff_mat = (-radi_skill + dire_skill).sub(
    sample_set.radi_adv_by_sample([-1]), 'rows')
radi_win_prob = 1 / (1 + np.exp(skill_diff_mat))
display(radi_win_prob.shape)
radi_win_prob.mean()

(1, 268)

matchId
4904311356    0.177271
4904261164    0.796778
4904218149    0.177346
4904171017    0.796619
4904062522    0.395269
4903974414    0.395441
4903679968    0.612977
4903566049    0.347967
4903441151    0.612724
4903243468    0.658495
4903157391    0.516036
4903091581    0.304947
4903051772    0.516179
4902924459    0.516190
4902837041    0.516331
4902729928    0.516607
4902576905    0.636197
4902464697    0.325951
4902354217    0.636013
4902186317    0.229738
4902122478    0.738891
4902056634    0.229906
4901998619    0.133437
4901925673    0.133479
4901835876    0.401121
4901799568    0.924753
4901742097    0.401212
4901733286    0.064316
4901728834    0.798126
4901646019    0.176242
                ...   
4890122311    0.292441
4889995626    0.075885
4889981518    0.879931
4889881001    0.549879
4889865830    0.760904
4889722230    0.336780
4889691529    0.415315
4889635707    0.473449
4889610539    0.326719
4889555136    0.612461
4889525518    0.170257
4889502467    0.625174
488

In [35]:
matches_ti9.assign(win_prob=radi_win_prob.iloc[-1]).loc[
    :, ["radiant_name", "dire_name", "win_prob", "radiantVictory"]]

,radiant_name,dire_name,win_prob,radiantVictory
matchId,,,,
4904311356,J.Storm,Forward Gaming,0.177271,False
4904261164,Forward Gaming,J.Storm,0.796778,True
4904218149,J.Storm,Forward Gaming,0.177346,False
4904171017,Forward Gaming,J.Storm,0.796619,False
4904062522,beastcoast,J.Storm,0.395269,False
4903974414,beastcoast,J.Storm,0.395441,False
4903679968,Chaos Esports Club,The Final Tribe,0.612977,True
4903566049,The Final Tribe,Chaos Esports Club,0.347967,False
4903441151,Chaos Esports Club,The Final Tribe,0.612724,True


### `GPSampleSet.radi_adv_by_sample()`

In [38]:
sample_set.radi_adv_by_sample()[-1]

-0.0842654745433928

## How long does it take to fit the full model?

In [40]:
players_mat = munge.make_match_players_matrix(
    matches.radiant_players, matches.dire_players)
players_mat.shape

(5000, 975)

In [41]:
all_players = src.munge.match_df_to_player_df(matches)

In [42]:
gp = src.models.gp.SkillsGP(
    players_mat,
    matches.startTimestamp.values,
    matches.radiantVictory,
    all_players.loc[players_mat.columns].name,
    "exponential",
    {"scale": two_years_in_ms},
    propose_sd=0.1,
    radi_prior_sd=10,  # Non-informative prior
    logistic_scale=1,
    save_every_n_iter=10
)

In [48]:
full_res = gp.fit()

Starting...
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Iteration.
Optimization terminated successfully.
         Current function value: -140678.649538
         Iterations: 15
         Function evaluations: 16
         Gradient evaluations: 30
         Hessian evaluations: 16194


In [59]:
skills_by_player = [full_res.x[start:end]
                    for start, end in gp.samples.skill_vec_idx_of_player]
gp.samples.samples.drop(0, inplace=True)
gp.samples.add_sample(0, skills_by_player, full_res.x[-1], full_res.fun)

In [60]:
full_sample_set = gp.samples

In [65]:
player_skill_by_sample = full_sample_set.player_skill_by_sample("pieliedie")
player_skill_by_sample

matchId,4904311356,4904261164,4904218149,4904171017,4901998619,4901925673,4901799568,4901733286,4899990684,4899794936,...,3800120344,3799989013,3799869808,3798272648,3798116566,3797921919,3797779580,3797616005,3796128008,3796013772
iter,,,,,,,,,,,,,,,,,,,,,
0,0.526078,0.526075,0.526045,0.525979,0.526674,0.526681,0.526655,0.526632,0.525757,0.52557,...,0.589754,0.589735,0.589776,0.591563,0.591662,0.591725,0.591702,0.591771,0.591635,0.591611


In [74]:
skills_by_player = pd.Series(
    [np.mean(full_sample_set.player_skill_by_sample(player).values)
     for player in full_sample_set.players.index],
    index=full_sample_set.players.index,
    name="skill")

In [125]:
temp = pd.DataFrame(skills_by_player).assign(name=all_players.name, team=all_players.team)\
    .sort_values("skill", ascending=False)
temp

,skill,name,team
94786276,1.047168,nine,J.Storm
121769650,1.039699,Nisha,Team Secret
119631156,1.026347,StiNGer,Infamous Gaming
182331313,1.014696,Fade,\tVici Gaming
34505203,0.992309,MinD_ContRoL,Team Liquid
73562326,0.965142,zai,Team Secret
311360822,0.893503,ana,OG
86727555,0.832631,ppd,Ninjas in Pyjamas
86700461,0.829591,w33,Team Liquid
114619230,0.810285,Crystallize,Natus Vincere


In [140]:
temp.loc[temp.team == "Alliance"]

,skill,name,team
86738694,0.796934,qojqva,Alliance
54580962,0.306711,iNSaNiA,Alliance
77490514,0.306711,Boxi,Alliance
401792574,0.202552,Taiga,Alliance
152962063,0.131486,miCKe,Alliance
95430068,0.093403,Madara,Alliance
101495620,-0.692069,Loda,Alliance


In [101]:
matches.loc[matches.dire_nicknames.apply(lambda x: "Newsham" in x)
            | matches.radiant_nicknames.apply(lambda x: "Newsham" in x)]

,dire_name,dire_nicknames,dire_players,dire_valveId,duration,league_id,league_name,radiantVictory,radiant_name,radiant_nicknames,radiant_players,radiant_valveId,seriesId,startDate,startTimestamp
matchId,,,,,,,,,,,,,,,
4668946745,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,2683,10646,ESL One Mumbai 2019,True,TNC Predator,"[Gabbi, TIMS, Armel, Eyyou, Kuku]","[152545459, 155494381, 164532005, 173476224, 1...",2108395,320721,2019-04-19 14:18:43,1555683523000
4668639784,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,2104,10646,ESL One Mumbai 2019,True,TNC Predator,"[Gabbi, TIMS, Armel, Eyyou, Kuku]","[152545459, 155494381, 164532005, 173476224, 1...",2108395,320721,2019-04-19 13:07:35,1555679255000
4664749507,Mineski,"[kpii, ninjaboogie, Febby, Moon, Ahjit]","[87012746, 91443418, 112377459, 113457795, 126...",543897,1737,10646,ESL One Mumbai 2019,False,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,320332,2019-04-18 11:21:30,1555586490000
4664556753,Mineski,"[kpii, ninjaboogie, Febby, Moon, Ahjit]","[87012746, 91443418, 112377459, 113457795, 126...",543897,2157,10646,ESL One Mumbai 2019,False,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,320332,2019-04-18 10:13:09,1555582389000
4658613378,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,1498,10646,ESL One Mumbai 2019,False,TNC Predator,"[Gabbi, TIMS, Armel, Eyyou, Kuku]","[152545459, 155494381, 164532005, 173476224, 1...",2108395,319734,2019-04-16 13:17:30,1555420650000
4658247555,TNC Predator,"[Gabbi, TIMS, Armel, Eyyou, Kuku]","[152545459, 155494381, 164532005, 173476224, 1...",2108395,1942,10646,ESL One Mumbai 2019,True,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,319734,2019-04-16 12:14:14,1555416854000
4657384268,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,4503,10646,ESL One Mumbai 2019,False,Natus Vincere,"[Misha, Crystallize, Chuvash, MagicaL, Blizzy]","[90125566, 114619230, 117483894, 171981096, 23...",36,319701,2019-04-16 07:48:11,1555400891000
4657218941,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,2570,10646,ESL One Mumbai 2019,False,Natus Vincere,"[Misha, Crystallize, Chuvash, MagicaL, Blizzy]","[90125566, 114619230, 117483894, 171981096, 23...",36,319701,2019-04-16 06:34:20,1555396460000
4641233076,Forward Gaming,"[Sneyking, MSS, YawaR, kitrak, CCnC]","[10366616, 86726887, 108452107, 187824320, 221...",6214538,2182,10725,ESL Birmingham 2019,False,TEAM TEAM,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",6904594,317858,2019-04-11 23:04:06,1555023846000


In [146]:
team_skills = (all_players.assign(skill=skills_by_player)
               .sort_values("skill")
               .groupby("team").agg({"skill": lambda x: sum(x[-5:])})
               .sort_values("skill", ascending=False))
with pd.option_context('max_rows', 1000):
    display(team_skills)

,skill
team,
Virtus.pro,3.645391
Team Secret,3.636711
Team Liquid,3.260040
PSG.LGD,3.196365
Forward Gaming,2.983200
\tVici Gaming,2.949907
OG,2.843128
Evil Geniuses,2.737325
Fnatic,2.705568


In [150]:
upper = np.triu(squareform(pdist(team_skills[:20], metric=lambda x, y: 1 / (1 + np.exp((y - x))))))
lower = np.tril(squareform(pdist(team_skills[:20], metric=lambda x, y: 1 / (1 + np.exp((x - y))))))

pd.DataFrame(
    upper + lower,
    index=team_skills.index[:20],
    columns=team_skills.index[:20]).iloc[:15, :15]

team,Virtus.pro,Team Secret,Team Liquid,PSG.LGD,Forward Gaming,Vici Gaming,OG,Evil Geniuses,Fnatic,Royal Never Give Up,Natus Vincere,Gambit Esports,TNC Predator,EHOME,Infamous Gaming
team,,,,,,,,,,,,,,,
Virtus.pro,0.000000,0.502170,0.595163,0.610407,0.659752,0.667186,0.690458,0.712604,0.719064,0.721207,0.722138,0.734246,0.781887,0.789443,0.797600
Team Secret,0.497830,0.000000,0.593070,0.608341,0.657801,0.665256,0.688600,0.710823,0.717307,0.719459,0.720393,0.732549,0.780403,0.787997,0.796195
Team Liquid,0.404837,0.406930,0.000000,0.515913,0.568771,0.576918,0.602744,0.627782,0.635173,0.637633,0.638704,0.652699,0.709168,0.718335,0.728299
PSG.LGD,0.389593,0.391659,0.484087,0.000000,0.553090,0.561305,0.587403,0.612787,0.620294,0.622796,0.623884,0.638128,0.695862,0.705275,0.715518
Forward Gaming,0.340248,0.342199,0.431229,0.446910,0.000000,0.508322,0.534961,0.561161,0.568966,0.571572,0.572707,0.587608,0.648968,0.659121,0.670218
\tVici Gaming,0.332814,0.334744,0.423082,0.438695,0.491678,0.000000,0.526670,0.552946,0.560783,0.563401,0.564540,0.579517,0.641347,0.651602,0.662818
OG,0.309542,0.311400,0.397256,0.412597,0.465039,0.473330,0.000000,0.526426,0.534336,0.536981,0.538133,0.553300,0.616433,0.626986,0.638555
Evil Geniuses,0.287396,0.289177,0.372218,0.387213,0.438839,0.447054,0.473574,0.000000,0.507939,0.510596,0.511755,0.527026,0.591128,0.601928,0.613796
Fnatic,0.280936,0.282693,0.364827,0.379706,0.431034,0.439217,0.465664,0.492061,0.000000,0.502659,0.503817,0.519104,0.583431,0.594295,0.606241


### Radiant advantage in the full dataset.

In [151]:
full_res.x[-1]

0.1780694409536598

In [153]:
src.models.gp.win_prob(full_res.x[-1], 1)

0.5444000996453496